In [ ]:
!apt-get install unrar


In [ ]:
!unrar x /content/colab.rar /content/


In [ ]:
#Data Augmenation

import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image, ImageEnhance, ImageOps, ImageFilter
import random

# Paths for directories
aug_train_dir = '/content/colab/Train/Train/Augmented'
aug_test_dir = '/content/colab/Test/Test/Augmented'

# Subfolders corresponding to the classes
subfolders = ['BG', 'N', 'D', 'P', 'V', 'S']

# Original dataset size
train_counts = {'BG': 75, 'D': 139, 'N': 75, 'P': 100, 'S': 122, 'V': 185}
test_counts = {'BG': 25, 'D': 46, 'N': 25, 'P': 34, 'S': 42, 'V': 62}

# Target counts for augmentation
train_target_count = 1500  # Target count for training set
test_target_count = 500    # Target count for test set

# Function to create augmentation folders
def create_aug_folders(base_dir):
    for subfolder in subfolders:
        aug_folder_path = os.path.join(base_dir, subfolder)
        os.makedirs(aug_folder_path, exist_ok=True)

# Create folders for augmented images
create_aug_folders(aug_train_dir)
create_aug_folders(aug_test_dir)

datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2
)

def augment_image(image):
    augmented_images = []
    # Horizontal and Vertical Flips
    augmented_images.append(ImageOps.mirror(image))  # Horizontal flip
    augmented_images.append(ImageOps.flip(image))  # Vertical flip
    # Vertical-Horizontal Flip
    augmented_images.append(ImageOps.mirror(ImageOps.flip(image)))
    # Gaussian Blur
    augmented_images.append(image.filter(ImageFilter.GaussianBlur(radius=2)))
    # Gaussian Noise
    noise = np.random.normal(0, 25, (image.size[1], image.size[0], 3)).astype(np.uint8)
    noisy_image = np.array(image) + noise
    augmented_images.append(Image.fromarray(np.clip(noisy_image, 0, 255).astype(np.uint8)))
    # Gamma Correction
    gamma_corrected = ImageEnhance.Brightness(image).enhance(1.5)
    augmented_images.append(gamma_corrected)
    # Equalize Histogram
    equalized_image = ImageOps.equalize(image)
    augmented_images.append(equalized_image)
    # Increase Saturation, Hue, Red, Green, Blue
    augmented_images.append(ImageEnhance.Color(image).enhance(1.5))  # Saturation
    # Adjust RGB channels
    for i, factor in enumerate([1.2, 1.1, 1.3]):
        channels = list(image.split())
        channels[i] = ImageEnhance.Brightness(channels[i]).enhance(factor)
        augmented_images.append(Image.merge(image.mode, channels))
    return augmented_images

# Function to apply augmentations and save
def apply_augmentations(image_path, save_dir, num_augmentations):
    # Load image
    img = load_img(image_path)
    img_array = img_to_array(img)
    generated_count = 0

    # Generate augmented images with ImageDataGenerator
    img_batch = np.expand_dims(img_array, axis=0)
    for batch in datagen.flow(img_batch, batch_size=1):
        aug_img = batch[0].astype(np.uint8)
        aug_img = Image.fromarray(aug_img)
        aug_img.save(os.path.join(save_dir, f"aug_{generated_count}_{os.path.basename(image_path)}"))
        generated_count += 1
        if generated_count >= num_augmentations:
            break

    if generated_count < num_augmentations:
        # Apply custom augmentations
        custom_augmented_images = augment_image(img)
        for aug_img in custom_augmented_images:
            if generated_count >= num_augmentations:
                break
            aug_img.save(os.path.join(save_dir, f"custom_aug_{generated_count}_{os.path.basename(image_path)}"))
            generated_count += 1

# Function to augment dataset
def augment_dataset(original_dir, aug_dir, counts, target_count):
    for subfolder in subfolders:
        folder_path = os.path.join(original_dir, subfolder)
        save_path = os.path.join(aug_dir, subfolder)
        os.makedirs(save_path, exist_ok=True)

        current_count = counts[subfolder]
        num_to_generate = target_count - current_count

        if num_to_generate <= 0:
            continue

        images = [f for f in os.listdir(folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]
        random.shuffle(images)

        for img_file in images:
            img_path = os.path.join(folder_path, img_file)
            apply_augmentations(img_path, save_path, num_to_generate // len(images))
            print(f"Augmented {img_file} from {subfolder} and saved to {save_path}")

        # Fill remaining images if needed
        remaining = target_count - len(os.listdir(save_path))
        while remaining > 0:
            for img_file in images:
                if remaining <= 0:
                    break
                img_path = os.path.join(folder_path, img_file)
                apply_augmentations(img_path, save_path, 1)
                remaining -= 1

# Augment train and test data
augment_dataset(
    '/content/colab/Train/Train',
    aug_train_dir,
    train_counts,
    train_target_count
)

augment_dataset(
    '/content/colab/Test/Test',
    aug_test_dir,
    test_counts,
    test_target_count
)


KeyboardInterrupt: 

In [ ]:
#VGG19

mport os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Paths to your train and test directories
train_dir = '/content/colab/Train/Train/Augmented'
test_dir = '/content/colab/Test/Test/Augmented'


# Image dimensions and batch size
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Normalizing the training and test datasets
datagen = ImageDataGenerator(rescale=1.0/255)


# Creating data generators
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load the VGG19 model with pre-trained weights from ImageNet
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Freezing the base model layers to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Adding custom layers on top of the base model
# x = base_model.output
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.5)(x)
# predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# x = base_model.output
# x = Flatten()(x)
# x = Dense(256, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)
# x = Dense(128, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)
# predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

from tensorflow.keras.regularizers import l2

# Adding L2 regularization to Dense layers
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)  # Add L2 here
x = BatchNormalization()(x)
x = Dropout(0.7)(x)

x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)  # Add L2 here
x = BatchNormalization()(x)
x = Dropout(0.7)(x)

predictions = Dense(6, activation='softmax')(x)


# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks: ModelCheckpoint and EarlyStopping
checkpoint = ModelCheckpoint("best_vgg19_model.keras", save_best_only=True, monitor="val_loss", mode="min", verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)

# Check if a saved model exists and resume training if available
if os.path.exists("/content/best_vgg19_model.keras"):
    # Load the saved model
    model = tf.keras.models.load_model("best_vgg19_model.keras")
    print("Model loaded from checkpoint.")
else:
    print("No saved model found, starting from scratch.")

# Train the model
EPOCHS = 20

steps_per_epoch = len(train_generator)
validation_steps = len(test_generator)
print(steps_per_epoch)
print(validation_steps)

# Callbacks: ReduceLROnPlateau, ModelCheckpoint, and EarlyStopping
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',   # Metric to monitor
    factor=0.5,           # Reduce learning rate by a factor of 0.5
    patience=3,           # Wait for 3 epochs of no improvement
    min_lr=1e-6,          # Minimum learning rate
    verbose=1             # Print updates
)

# Callbacks list
callbacks = [checkpoint, early_stopping, reduce_lr]

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=callbacks,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy (initial): {test_acc:.2f}")




# Fine-tuning: Unfreeze some of the top layers of the base model
for layer in base_model.layers[-12:]:  # Adjust the number of layers to unfreeze
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


# Additional training for fine-tuning
fine_tune_epochs = 10
total_epochs = EPOCHS + fine_tune_epochs

fine_tune_history = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=test_generator,
    callbacks=callbacks,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy after fine-tuning: {test_acc:.2f}")



# Save the model
# model.save("vgg19_finetuned_model.h5")


In [ ]:
#densenet201

import os
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Path to your augmented dataset
train_dir = '/content/colab/Train/Train/Augmented'
val_dir = '/content/colab/Test/Test/Augmented'

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Input size for DenseNet201
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),  # Input size for DenseNet201
    batch_size=32,
    class_mode='categorical'
)

# Build DenseNet201 Model
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)  # Adjust output classes

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Save the model
model.save('densenet201_footulcer.h5')

print("Model training completed and saved as 'densenet201_footulcer.h5'")


In [ ]:
#mobilenetv2

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras import mixed_precision

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

# Paths to your train and test directories
train_dir = '/content/colab/Train/Train/Augmented'
test_dir = '/content/colab/Test/Test/Augmented'

# Image dimensions and batch size
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Data augmentation for training and normalization for testing
datagen = ImageDataGenerator(
    rescale=1.0/255
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Data generators
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load the MobileNetV2 base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Custom classification head with BatchNormalization and L2 regularization
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks: ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint("best_mobilenet_model.keras", save_best_only=True, monitor="val_loss", mode="min", verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Check for saved model
if os.path.exists("best_mobilenet_model.keras"):
    model = tf.keras.models.load_model("best_mobilenet_model.keras")
    print("Model loaded from checkpoint.")
else:
    print("No saved model found, starting from scratch.")

# Train the model
EPOCHS = 20

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy (initial): {test_acc:.2f}")

In [ ]:
#NasNet

import tensorflow as tf
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import os

# Paths to your dataset
train_dir = '/content/colab/Train/Train/Augmented'
test_dir = '/content/colab/Test/Test/Augmented'

# Paths for saving the model
model_path = 'best_model.keras'

# Parameters
img_size = 224  # NASNetMobile requires 224x224 input
batch_size = 32
num_classes = 6
epochs = 10

# Data Augmentation and Loading
train_datagen = ImageDataGenerator(
    rescale=1.0/255
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Check if a pre-trained model exists
if os.path.exists(model_path):
    print("Loading existing model...")
    model = tf.keras.models.load_model(model_path)
else:
    # Load Pretrained NASNetMobile
    base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

    # Freeze the base model
    base_model.trainable = False

    # Add custom layers with Dropout and L2 Regularization
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensions
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)  # L2 regularization
    x = Dropout(0.5)(x)  # Dropout layer
    predictions = Dense(num_classes, activation='softmax')(x)  # Final output layer

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Callbacks: ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-7)

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

# Save the final model
model.save('nasnet_transfer_learning_final_model.keras')


In [ ]:
#Evalutation results (run for each saved model)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    roc_auc_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

# Path to your validation dataset
val_dir = '/content/colab/Test/Test/Augmented'

# Load validation data
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),  # DenseNet201 default input size
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Ensure no shuffling for proper evaluation
)

def evaluate_and_visualize(model_path, data_generator):
    """
    Evaluates and visualizes the DenseNet201 model's performance.

    Parameters:
    - model_path (str): Path to the saved DenseNet201 model.
    - data_generator: The ImageDataGenerator for the test/validation data.

    Returns:
    - A dictionary with accuracy, precision, recall, and F1-score.
    """
    try:
        # Load the saved DenseNet201 model
        model = load_model(model_path)

        # Get true labels and predict probabilities
        y_true = data_generator.classes  # True class labels
        y_pred_prob = model.predict(data_generator, verbose=1)  # Predicted probabilities

        # Convert probabilities to predicted class indices
        y_pred = np.argmax(y_pred_prob, axis=1)

        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='macro')
        recall = recall_score(y_true, y_pred, average='macro')
        f1 = f1_score(y_true, y_pred, average='macro')

        # Classification report
        print("Classification Report:\n", classification_report(y_true, y_pred, target_names=list(data_generator.class_indices.keys())))

        # Visualize metrics
        metrics = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1-score": f1}
        visualize_metrics(metrics)

        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        visualize_confusion_matrix(cm, data_generator.class_indices.keys())

        # ROC Curve
        visualize_roc_curve(y_true, y_pred_prob, data_generator.class_indices.keys())

        # Return the metrics
        return metrics
    except Exception as e:
        print(f"An error occurred during evaluation: {e}")
        return None

def visualize_metrics(metrics):
    """
    Plots a bar chart for the provided metrics.

    Parameters:
    - metrics (dict): A dictionary with metric names as keys and values as scores.
    """
    names = list(metrics.keys())
    values = list(metrics.values())

    plt.figure(figsize=(8, 5))
    plt.bar(names, values, color=['skyblue', 'orange', 'green', 'red'])
    plt.ylim(0, 1)  # Assuming metrics are between 0 and 1
    plt.ylabel("Score")
    plt.title("Model Performance Metrics")
    for i, value in enumerate(values):
        plt.text(i, value + 0.02, f"{value:.2f}", ha='center', fontsize=12)
    plt.show()

def visualize_confusion_matrix(cm, class_labels):
    """
    Visualizes the confusion matrix.

    Parameters:
    - cm (ndarray): The confusion matrix.
    - class_labels: The list of class labels.
    """
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
    disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
    plt.title("Confusion Matrix")
    plt.show()

def visualize_roc_curve(y_true, y_pred_prob, class_labels):
    """
    Plots the ROC Curve for each class.

    Parameters:
    - y_true (array): True class labels.
    - y_pred_prob (array): Predicted probabilities.
    - class_labels (list): List of class labels.
    """
    plt.figure(figsize=(10, 8))
    for i, label in enumerate(class_labels):
        fpr, tpr, _ = roc_curve(y_true == i, y_pred_prob[:, i])
        auc = roc_auc_score(y_true == i, y_pred_prob[:, i])
        plt.plot(fpr, tpr, label=f"{label} (AUC = {auc:.2f})")
    plt.plot([0, 1], [0, 1], 'k--')  # Random chance line
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.show()

# Example usage
model_path = 'best_model.keras'  # Update with your DenseNet201 model file path
results = evaluate_and_visualize(model_path, val_generator)

if results:
    print(f"Metrics:")
    for metric, value in results.items():
        print(f"{metric.capitalize()}: {value:.2f}")
